In [7]:
import pandas as pd

In [9]:
data = pd.read_csv('extra_qa - Sheet1.csv')

In [10]:
data

,Question,Answer
0,How to diagnose Asthma?,Episodic symptoms of airflow obstruction are p...
1,How to treat Asthma ?,\nPharmacologic management includes the use of...
2,What are the signs and symptoms of asthma?,Signs and symptoms of asthma include the follo...
3,Which medications are used in the treatment of...,Pharmacologic management includes the use of r...
4,What is the anatomical structure of the airway...,The airways of the lungs consist of the cartil...
...,...,...
62,Which symptoms of sinus disease are also assoc...,"Sinus disease, especially in children, is asso..."
63,What is the relationship between gastroesophag...,"Cough, recurrent bronchitis, pneumonia, wheezi..."
64,Which conditions may be mistaken for asthma?,"Other extrinsic conditions, such as lymphadeno..."
65,Which factor distinguishes COPD from asthma?,A significant history of smoking greater than ...


In [12]:
questions_only = data['Question'].values
context_only = data['Answer'].values

In [13]:
from flair.data import Sentence
from flair.models import SequenceTagger

/local/home/tchimurkar/anaconda3/envs/sail/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [14]:
tagger = SequenceTagger.load("flair/pos-english")

2022-05-04 17:35:50,308 loading file /local/home/tchimurkar/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63
2022-05-04 17:35:50,509 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [15]:
import numpy as np

all_texts_combined = np.concatenate([questions_only, context_only])

In [16]:
all_text_df = pd.DataFrame(all_texts_combined)

In [17]:
all_text_df.dropna(inplace=True)
all_text_df.shape

(133, 1)

In [21]:
all_text_df

,0
0,How to diagnose Asthma?
1,How to treat Asthma ?
2,What are the signs and symptoms of asthma?
3,Which medications are used in the treatment of...
4,What is the anatomical structure of the airway...
...,...
129,"Sinus disease, especially in children, is asso..."
130,"Cough, recurrent bronchitis, pneumonia, wheezi..."
131,"Other extrinsic conditions, such as lymphadeno..."
132,A significant history of smoking greater than ...


In [18]:
all_text_df.drop_duplicates(inplace=True)
all_text_df.shape

(133, 1)

In [22]:
all_text_df.columns = ['texts']
all_texts_combined = all_text_df['texts'].tolist()

In [23]:
all_texts_combined_single = " ".join(all_texts_combined)

In [24]:
pos_tag_json = {}

def get_pos_tags(sentence, pos_tag_json = {}):
    tag = sentence.labels[0].value
    word = sentence.text
    if tag in pos_tag_json.keys():
        pos_tag_json[tag].append(word)
    else:
        pos_tag_json[tag] = []
        pos_tag_json[tag].append(word)
    return pos_tag_json

In [25]:
from tqdm import tqdm

tqdm.pandas()

In [26]:
pos_json = {}
for xi in tqdm(all_texts_combined):
    sentence = Sentence(xi)
    tagger.predict(sentence)
    for elem in sentence:
        pos_json = get_pos_tags(elem, pos_json)

100%|██████████| 133/133 [00:14<00:00,  8.88it/s]


In [27]:
pos_json_unq = {}
for key, value in pos_json.items():
    pos_json_unq[key] = list(set(value))

In [28]:
pos_tags_after_115 = pd.DataFrame.from_dict(pos_json_unq, orient='index')

In [30]:
pos_tags_after_115.to_csv('pos_tags_115_after.csv')